In [ ]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime
from ..extras import helpers, create_table_leanx as ctl

from ..masterdata import values,master_data, text_data
import inventory_doc_data

In [ ]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.sales_doc_types, 
    text_data.sales_organizations,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons,
    text_data.releases,
    text_data.purchasing_doc_types,
    text_data.purchasing_organizations
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        all_cols_req = ctl.clean_columns(all_cols,table_name)
        df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
        df.to_csv(f'data/inv/ocpm/text/{table_name}.csv', index=False)

In [ ]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        all_cols_req = ctl.clean_columns(all_cols,table_name)
        df = pd.concat([all_cols_req, pd.DataFrame(v.values())])
        df.to_csv(f'data/inv/ocpm/master/{table_name}.csv', index=False)

In [ ]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

In [ ]:
MARC = pd.read_csv('data/P2P/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/P2P/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/P2P/OCPM/master/MAKT.csv')
LFB1 = pd.read_csv('data/P2P/OCPM/master/LFB1.csv')
LFA1 = pd.read_csv('data/P2P/OCPM/master/LFA1.csv')

all_prices = {}
all_has_contract_probabilities = {}
all_matnr_names = {}
all_availabilities = {}

# get price and material group by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_matnr_names[nr] = mat
                    all_has_contract_probabilities[nr] = attr['has_contract_probability']
                    all_availabilities[nr] = attr['availability']

In [ ]:
def get_params():
    # basic informaiton
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = str(uuid.uuid4())[-15:]
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    vendor_name = LFA1[LFA1['LIFNR'] == lifnr]['NAME1'].values[0]
    purchasing_org=random.choice(values.om_plants[plant]['purchasing_orgs'])
    new_lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    requested_by = get_user_name(0.1)

    # materials related
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))

    # quantities
    quantity_factor = random.randint(24, 48) if values.om_plants[plant]['high_value'] else random.randint(6, 18)
    quantities = [random.randint(12, 60)*quantity_factor for temp_quantity_index in range(len(matnrs))]
    has_qty_change = True if values.om_company_codes[company_code]['incorrect_qty_prbobability'] > 0.3 else False

    # prices and contract
    prices = [all_prices[matnrs[temp_matnr_index]] for temp_matnr_index in range(len(matnrs))]
    has_contract_probabilities = [all_has_contract_probabilities[matnrs[temp_contract_index]] for temp_contract_index in range(len(matnrs))]
    item_has_contract = [True if random.random() < has_contract_probabilities[j] else False for j in range(len(has_contract_probabilities))]
    has_contract = True 
    for hc in item_has_contract:
        if hc == False:
            has_contract = False
            break
    
    # delivery status
    delivery_status = [] # early: -x | on_time: 0 | late: x
    for temp_delivery_status_index in range(len(matnrs)):
        extra_days = 0
        if lifnr in ('VND1','VND6','VND14') and random.random() > 0.2:
             extra_days = random.randint(2, 14) # late
        elif all_availabilities[matnrs[temp_delivery_status_index]] < 0.3: 
            extra_days = random.randint(2, 14) # late
        elif (all_availabilities[matnrs[temp_delivery_status_index]] >= 0.3) and (all_availabilities[matnrs[temp_delivery_status_index]] < 0.8):
            extra_days = 0 # on time
        else:
            extra_days = -random.randint(2, 5)
        delivery_status.append(extra_days)
        
    # free text
    is_free_text = True if random.random() < values.om_company_codes[company_code]['free_text_pr_probability'] else False
    free_text_materials = [all_matnr_names[matnr] for matnr in matnrs] # TODO change with material description

    # time distribution
    pr_req_years = [2022, 2023]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_month_distribution = np.array([7, 6, 5, 4, 2, 2, 3, 5, 5, 6, 7, 10])
    pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
    pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]

    params = {
        'matnrs': matnrs,
        'konnr': konnr,
        'lifnr': lifnr,
        'plant': plant,
        'quantities': quantities,
        'prices': prices, # considered contract and PO prices
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        
        'item_has_contract': item_has_contract,
        'has_contract': has_contract,
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,

        'has_qty_change': has_qty_change,

        'has_payment_term_change': random.random() > values.proc_vendors[vendor_name]['payment_term_stsability'],
        'delivery_status': delivery_status,

        'new_payment_term': 'Z090',
        'new_vendor': new_lifnr,
        
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
    }

    return params

In [ ]:
inventory_doc_tables = {
    'EBAN_json': {}, 
	'CDHDR_json': {},
	'CDPOS_json': {},
	'EKKO_json': {},
	'EKPO_json': {},
	'NAST_json': {},
	'MSEG_json': {},
	'EKBE_json': {},
	'RBKP_json': {},
	'RSEG_json': {},
	'EKET_json': {},
}
# for i in range(1000):
# # for i in range(1_000):
# 	transition_prob = np.array([
# 	  # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ] 
# 		[0.00, 0.50, 1.30, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 0 Create PO
# 		[0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 1 Approve PO
# 		[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 2 Send PO
# 		[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.01], # 3 Post Goods Receipt
# 		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 4 Create Vendor Invoice

# 		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 5 Change Payment Term
# 		[0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.15], # 6 Change Vendor
# 		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 7 Change Quantity

# 		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 8 Terminate
# 	])
# 	"""
# 	0: Create PO
# 	1: Approve PO
# 	2: Send PO
# 	3: Post GR
# 	4: Create Vendor Invoice

# 	5: Change Payment Term
# 	6: Change Vendor
# 	7: Change Quantity
	
# 	8: Terminate
# 	"""

#INVENTORY MANAGEMENT EVENTS LIST
for i in range(500):
# for i in range(1_000):
	transition_prob = np.array([
	  # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ] 
		[0.00, 0.50, 1.30, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 0 Create SalesOrder
		[0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 1 Create ProductionOrderHeader
		[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 2 Create PurchaseOrderItem
		[0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.15], # 3 Post Goods Receipt
		[0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.01], # 4 Begin ProductionOrder
		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 5 Finish ProductionOrder
		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 6 Post Goods Issue 
		[0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 7 Reverse Goods Issue

		[0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 8 Terminate
	])

	
	params = get_params()
	latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
	latest_time = helpers.generate_random_time()
	inv = inventory_doc_data.Inventory(params=params, start_date=latest_date, index=i, vbeln = f'{str(uuid.uuid4())[-11:]}')
	preq_creation_date = latest_date

	pr_automation_rate = 0.2 if params['is_free_text'] else 0.7

	step = 0 # Create PO
	creation_time = helpers.generate_random_time()
	purchase_creation_date = latest_date
	total = sum([params['quantities'][j]*params['prices'][j] for j in range(len(params['matnrs']))])

	while step != 8:
		if step == 0:
			inv.create_sales_order(
			reqested_delivery_date=reqested_delivery_date,
			shipping_condition=random.choice(list(values.shipping_conditions.keys())),
			erdat=latest_date,
			ernam=so_created_by,
			atime=latest_time
    		)

		elif step == 1:
			inv.create_production_order_header(
				aedat=latest_date,
				ernam = get_user_name(0.2)
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 2:
			inv.create_purchase_order(
				aedat=latest_date,
				ernam=get_user_name(0.8),
				utime=creation_time
			)
			latest_date += helpers.UPTO_DAY()

		elif step == 3:
			inv.goods_receipt(
				cpudt=latest_date,
				usnam=get_user_name(0.7),
				atime=latest_time,
				udate=purchase_creation_date,
				utime=creation_time
			)
			latest_date += helpers.UPTO_WEEK()

		elif step == 4:
			inv.begin_production_order()

		elif step == 5:
			inv.finish_production_order( # likely goes next to 'Post GR'
				udate=latest_date,
				ernam=get_user_name(0.2)
			)
			latest_date += helpers.UPTO_MONTH()

		elif step == 6:
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(3, 6, latest_date)
            inv.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.6*params['automation_rate']),
                delivery_date_deviation = delivery_date_deviation,
                atime=helpers.generate_random_time()         
            )

		elif step == 7:
			inv.reverse_goods_issue()
			latest_date += helpers.UPTO_WEEK()
		
		
		# normalize transition matrix
		transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
		step = np.random.choice(9, p=transition_prob[step])

	for k, v in inventory.tables.items():
		for entry_key in list(v.keys()):
			inventory_doc_tables[k][entry_key] = inventory.tables[k][entry_key]

In [ ]:
for table, rows in inventory_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])
    df.to_csv(f'data/inv/ocpm/inventory-doc/{table_name}.csv', index=False)